In [136]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample
from collections import Counter
import string
from sklearn.dummy import DummyClassifier
from numpy.random import random_integers
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB

In [2]:
def readTxt(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep='\t')
    
    return df

In [3]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep=',')
    
    return df

In [59]:
def toCsvDf(fileName, df):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df.to_csv(fullFileName, header = False, sep='\t', index = False)

In [58]:
def toCsvNp(fileName, npArray):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')

In [129]:
def createDictionary(trainingSet):
    allWords = list()
    cnt = Counter()
    
    if trainingSet=='IMDB':
        trainingDF = readTxt('\IMDB-train.txt')
    elif trainingSet=='yelp':
        trainingDF = readTxt('\yelp-train.txt')
        
    translator = str.maketrans('','', string.punctuation)
    
    for i in range(0, len(trainingDF)):
        allWords.extend(trainingDF.iloc[i,0].translate(translator).lower().split(" "))
    
    for word in allWords:
        cnt[word] +=1
    
    dictionaryWords = list(zip(*cnt.most_common(10000)))[0]
    
    dictionary = {}
    for index, key in enumerate(dictionaryWords):
        dictionary[key] = index
#     print(dictionary)
    newArray = np.asarray(cnt.most_common(10000))
    withIndexVocabArray = np.insert(newArray, 1, range(0,10000),1)
    
#     if trainingSet=='IMDB':
#         toCsvDf('\IMDB-vocab.txt', pd.DataFrame(withIndexVocabArray))
#     elif trainingSet=='yelp':
#         toCsvDf('\yelp-vocab.txt', pd.DataFrame(withIndexVocabArray))
        
    return dictionary

In [76]:
def convertWordsToVector(trainingRow, dictionary):
    translator = str.maketrans('','', string.punctuation)
    returnRow = trainingRow.translate(translator).lower().split(" ")
    vector = np.zeros(10000, dtype=np.int8)
    
    for word in returnRow:
        if word in dictionary:
            vector[dictionary[word]] = np.int8(1)
            
    return vector

In [131]:
def createBagOfWordsMatrix(df, dictionary):
    translator = str.maketrans('','', string.punctuation)
    
    distancesArray = np.zeros((len(df), len(dictionary)), dtype=np.int8)
        
    for i in range(0, len(df)):
        vector = convertWordsToVector(df.iloc[i,0], dictionary)
        distancesArray[i] = vector

    return distancesArray
# trainingDF = readTxt('\IMDB-train.txt')
# npArray = createBagOfWordsMatrix('IMDB')
# print(npArray)
# dfArray = pd.DataFrame(npArray)
# dfArray.to_csv(r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets\TESTING2.csv', header = False, sep=',', index = False)

In [148]:
def randomClassifier(dataset):
    print(dataset, 'Random Classifier')
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
    
    randomClassifier = DummyClassifier(strategy='uniform')
    randomClassifier.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = randomClassifier.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = randomClassifier.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = randomClassifier.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
randomClassifier('yelp')

yelp Random Classifier
Training F1-Measure
F1 Score: 0.180419612228
Confusion Matrix: [[104 102 114  97 105]
 [115 134 136 124 132]
 [225 197 183 195 197]
 [490 496 481 487 514]
 [459 486 461 479 487]]
Validation F1-Measure
F1 Score: 0.169737750897
Confusion Matrix: [[15 16 15 22 16]
 [12 21 22 21 20]
 [47 33 24 28 32]
 [71 74 78 67 66]
 [69 72 47 55 57]]
Test F1-Measure
F1 Score: 0.186163692511
Confusion Matrix: [[ 38  31  28  25  21]
 [ 41  36  34  34  45]
 [ 60  48  58  76  58]
 [148 157 134 124 139]
 [119 149 124 125 148]]


In [150]:
def majorityClassifier(dataset):
    print(dataset, 'Majority Classifier')
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
    
    majorityClassifier = DummyClassifier(strategy='most_frequent')
    majorityClassifier.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = majorityClassifier.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = majorityClassifier.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = majorityClassifier.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
# baselineClassifier('IMDB')
majorityClassifier('yelp')

yelp Majority Classifier
Training F1-Measure


C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 Score: 0.104267004647
Confusion Matrix: [[   0    0    0  522    0]
 [   0    0    0  641    0]
 [   0    0    0  997    0]
 [   0    0    0 2468    0]
 [   0    0    0 2372    0]]
Validation F1-Measure
F1 Score: 0.105014749263
Confusion Matrix: [[  0   0   0  84   0]
 [  0   0   0  96   0]
 [  0   0   0 164   0]
 [  0   0   0 356   0]
 [  0   0   0 300   0]]
Test F1-Measure
F1 Score: 0.103923019985
Confusion Matrix: [[  0   0   0 143   0]
 [  0   0   0 190   0]
 [  0   0   0 300   0]
 [  0   0   0 702   0]
 [  0   0   0 665   0]]


In [140]:
def testScores(predicted, actual):
    print('F1 Score:', f1_score(actual, predicted, average='macro'))
    print('Confusion Matrix:', confusion_matrix(actual, predicted))

In [144]:
def bernoulliNB(dataset):
    print(dataset)
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
        
    clf = BernoulliNB()
    clf.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
bernoulliNB('yelp')

yelp
Training F1-Measure
F1 Score: 0.580313605691
Confusion Matrix: [[ 274    5   17   22  204]
 [   9  320   28   85  199]
 [  10   27  474  157  329]
 [  36  144   83 1237  968]
 [  50  121   75  247 1879]]
Validation F1-Measure
F1 Score: 0.326966475705
Confusion Matrix: [[ 27  12   5   6  34]
 [  9  17  11  27  32]
 [  4  21  28  55  56]
 [  9  29  27 119 172]
 [ 11  22  14  64 189]]
Test F1-Measure
F1 Score: 0.342970426948
Confusion Matrix: [[ 40  24   6  21  52]
 [ 20  47  33  29  61]
 [ 11  34  47  94 114]
 [  9  58  63 250 322]
 [ 10  38  24 153 440]]
